In [1]:
import numpy as np
import matplotlib.pyplot as plt
import batman, catwoman
from multiprocessing import Pool
import emcee

In [2]:
def calc_H(T, M, R, mm=2.3):
    """ Calculates the approximate scale height of a planet's atmosphere, using the equation
     scale height = kT / mg
    
    Inputs: T = the atmospheric temperature in [K]; M = the planet's mass in [kg]; 
            R = the planet's radius in [m]; mm = mean mass of a molecule in the atmosphere [amu], this is
                   default set to 1 amu = 1 proton mass (for now)
    Outputs: H = the scale height in [m]
    """
    # constants:
    amu = 1.67e-27 # [kg]; atomic mass unit in [kg]
    k = 1.38e-23 # [Joule/K]; Boltzmann constant
    G = 6.674e-11 # [m^3/kg/s^2]; Gravitational constant
    Mjupiter = 1.9e27 # [kg]; mass of Jupiter
    Rjupiter = 69911000.0 # [m]; approx. radius of Jupiter
    # computing the numerator for the scale height equation:
    E_thermal = k*T # [Joule]
    # computing the denominator:
    g = G*M/(R**2) # gravitational acceleration in [m/s^2]
    meanmass = mm*amu
    denominator = meanmass*g # [kg*m/s^2]
    # compute the scale height:
    H = E_thermal / denominator # [meters]
    return H

In [3]:
AU_to_meter = 1.496e11
day_to_second = 24.*60.*60.
Rsun_to_meter = 6.957e8
Rjup_to_meter = 7.149e7
Mjup_to_kg = 1.899e27

# create a planet
t0 = 0.  # [day]
P = 3.0  # orbital period [day]
aRs = 12.0  # semi-major axis in [stellar radii]
inc = 90.0 # orbital inclination[degrees]
phi = 90. # obliquity of planetary rotation axis [deg]
ecc = 0. # orbital eccentricity
w = 90.  # argument of periastron
rprs1 = 0.15 # evening-limb radius [stellar radii]
rprs2 = 0.15 # morning-limb radius [stellar radii]
rprs = np.sqrt(0.5*(rprs1**2 + rprs2**2)) # equivalent full-depth uniform radius [stellar radii]
Rs = 1.0 # stellar radius [Rsun]
Rp1 = rprs1*Rs # evening-limb radius [Rsun]
Rp2 = rprs2*Rs # morning-limb radius [Rsun]
Rp = rprs*Rs # equivalent full-depth uniform radius [Rsun]
Mp_jup = 0.281 # planet mass [jupiter masses]
Teq = 1166.0 # planet equilibrium temperature [K]
T14 = 3.0 # reference total transit duration [hours]

# calculate things in physical units
a_meter = aRs * Rs * Rsun_to_meter # semi-major axis in [m] ([Rstar] -> [Rsun] -> [m])
per_seconds = P*day_to_second # orbital period in [s]
Rs_meter = Rs * Rsun_to_meter # stellar radius in [m]
Rp_meter = Rp * Rsun_to_meter # planet radius in [m]
Rp1_meter = Rp1 * Rsun_to_meter
Rp2_meter = Rp2 * Rsun_to_meter
Mp_kg = Mp_jup * Mjup_to_kg # planet mass in [kg]

# calculate other things
# tangential circular orbital velocity, assuming Mp << Mstar
v_orb = (2.*np.pi*a_meter) / (per_seconds) # [m/s]
# atmospheric scale height, with bulk quantities
H = calc_H(Teq, Mp_kg, Rp_meter, mm=2.3) # [m]
print('H = %.1f km'%(H/1000.))
print(' = %.2f percent of Rp'%(100.*(H/Rp_meter)))

# create a high res. time axis for transit models
t_int = 0.2 # [s] per integration
obs_window_size = (0.75*T14) / 24. # half-duration of whole observation in [day]
t = np.arange(-obs_window_size, obs_window_size, (t_int / 60./60./24.))

H = 1281.0 km
 = 1.23 percent of Rp


In [4]:
## Generate transit models, all assuming no limb darkening
# with asymmetric limbs:
aparams  = catwoman.TransitParams()
aparams.t0 = t0            #time of inferior conjuction (in days)
aparams.per = P            #orbital period (in days)
aparams.rp = rprs1         #top semi-circle radius (in units of stellar radii)
aparams.rp2 = rprs2        #bottom semi-circle radius (in units of stellar radii)
aparams.phi = phi          #angle of rotation of top semi-circle (in degrees)
aparams.a = aRs            #semi-major axis (in units of stellar radii)
aparams.inc = inc          #orbital inclination (in degrees)
aparams.ecc = ecc          #eccentricity
aparams.w = w              #longitude of periastron (in degrees)
aparams.limb_dark = "uniform"  #limbs darkening model
asymmodel = catwoman.TransitModel(aparams,t)         #initalises model

# with uniform limbs
uparams  = batman.TransitParams()
uparams.t0 = t0             #time of inferior conjuction (in days)
uparams.per = P             #orbital period (in days)
uparams.rp = rprs           #top semi-circle radius (in units of stellar radii)
uparams.a = aRs             #semi-major axis (in units of stellar radii)
uparams.inc = inc           #orbital inclination (in degrees)
uparams.ecc = ecc           #eccentricity
uparams.w = w                #longitude of periastron (in degrees)
uparams.u = []               #limb darkening coefficients [u1, u2]
uparams.limb_dark = "uniform"   #limbs darkening model
unifmodel = batman.TransitModel(uparams,t)         #initalises model

In the simulations I've done to-date, I always start with the evening and morning limb radii at an initial value then inflate the morning limb radius by $\Delta R$. The morning limb is the leading limb. This always has the net effect of shifting $T_1$ earlier, but leaving $T_4$ as-is. Let's start by seeing if my analytic formula can predict the resulting $\Delta T_1$. 

Based on my formula, we should have
$$
|\Delta T_1| = |T_{1, inflated} - T_{1, uniform}| = \frac{\Delta R}{v_p} ~~~ (1)
$$

In [5]:
def predict_dT1(dR, vp):
    # calculates the expected shift in T1 based on eqn (1)
    # assumes dR in [m] and vp in [m/s]
    # returns dT1 in [s]
    dT = (dR / vp)
    return dT

def get_T1_index(time, y):
    # function to estimate the array index where T1 occurs, based on input model light curve
    # defined as where model flux first goes < 1
    pretransit_idxs = np.where((y == 1.0) & (time < np.median(time)))[0] # 'oot' indexes
    T1_idx = pretransit_idxs[-1] 
    return T1_idx

def get_T4_index(time, y):
    # get the index where T4 occurs
    # defined as where model flux last goes = 1
    posttransit_idxs = np.where((y == 1.0) & (time > np.median(time)))[0] # 'oot' indexes
    T4_idx = posttransit_idxs[0] 
    return T4_idx

So, I'll generate some catwoman model light curves. I'll start with uniform limbs, and record $T_1$ as best as I can, which will be $T_{1, uniform}$. Then, I'll increase the morning limb radius by $\Delta R$, and repeat to get $T_{1, inflated}$. Then, I'll compare their difference to the prediction from equation (1). By construction, should have $\Delta T_4$ = 0 too, which I'll check. 

In [6]:
## start with uniform radii
Rp_init = Rp1_meter # starting limb radii in [m]
rprs_init = Rp1_meter / Rs_meter
aparams.rp = rprs_init
aparams.rp2 = rprs_init
# generate the corresponding light curve
lc_uniform = asymmodel.light_curve(aparams)
# get the estimated first contact point
T1_uniform = t[get_T1_index(t, lc_uniform)]
# and last contact point
T4_uniform = t[get_T4_index(t, lc_uniform)]

## now inflate the morning limb radius
inflation = 1.5 # N scale heights by which to inflate radius
dR = inflation*H
Rp_inflated = Rp_init + dR
rprs_inflated = (Rp_inflated) / Rs_meter
aparams.rp = rprs_init
aparams.rp2 = rprs_inflated
# generate the corresponding light curve
lc_inflated = asymmodel.light_curve(aparams)
# get the estimated first contact point
T1_inflated = t[get_T1_index(t, lc_inflated)]
# and last contact point
T4_inflated = t[get_T4_index(t, lc_inflated)]

## calculate the prediction based on our analytic formula
dT1_predicted = abs(predict_dT1(dR, v_orb)) # [s]
dT4_predicted = 0. # [s]
# and what it is based on the models
dT1_models = abs(T1_inflated - T1_uniform)*24.*60.*60. # [s]
dT4_models = abs(T4_inflated - T4_uniform)*24.*60.*60. # [s]

In [7]:
print('T1 test -----')
print('Rp init = %.1f m'%(Rp_init))
print('dR = %.1f m (%.2f percent of Rp init)'%(dR, (dR/Rp_init)*100.))
print('inflated morning Rp = %.1f m'%(Rp_inflated))
print('---')
print('Predicted dT1 = %.2f s'%(dT1_predicted))
print('Modeled dT1 = %.2f s'%(dT1_models))
dT1_ratio = dT1_models / dT1_predicted
print('     Modeled / Predicted = %.3f'%(dT1_ratio))
print('Predicted dT4 = %.2f s'%(dT4_predicted))
print('Modeled dT4 = %.2f s'%(dT4_models))
dT4_ratio = dT4_models / dT4_predicted
print('     Modeled / Predicted = %.3f'%(dT4_ratio))

T1 test -----
Rp init = 104355000.0 m
dR = 1921470.1 m (1.84 percent of Rp init)
inflated morning Rp = 106276470.1 m
---
Predicted dT1 = 9.49 s
Modeled dT1 = 9.60 s
     Modeled / Predicted = 1.011
Predicted dT4 = 0.00 s
Modeled dT4 = 0.00 s
     Modeled / Predicted = nan


<ipython-input-7-ae2c45137bb2>:12: RuntimeWarning: invalid value encountered in double_scalars
  dT4_ratio = dT4_models / dT4_predicted


Testing different inflation percentages, I find that the analytical prediction gets it right within a percent each time. So this passes! Let's run it on a loop through different dR values and plot the result visually.

In [ ]:
def test_dT1(inflation):
    ## a function to automate the test parameters
    ## start with uniform radii
    Rp_init = Rp_meter # starting limb radii in [m]
    rprs_init = Rp_meter / Rs_meter
    aparams.rp = rprs_init
    aparams.rp2 = rprs_init
    # generate the corresponding light curve
    lc_uniform = asymmodel.light_curve(aparams)
    # get the estimated first contact point
    T1_uniform = t[get_T1_index(t, lc_uniform)]
    # and last
    T4_uniform = t[get_T4_index(t, lc_uniform)]

    ## now inflate the morning limb radius
    #inflation is an input # % by which to increase radius, relative to initial radius
    dR = inflation*H
    Rp_inflated = Rp_init + dR
    rprs_inflated = (Rp_inflated) / Rs_meter
    aparams.rp = rprs_init
    aparams.rp2 = rprs_inflated
    # generate the corresponding light curve
    lc_inflated = asymmodel.light_curve(aparams)
    # get the estimated first contact point
    T1_inflated = t[get_T1_index(t, lc_inflated)]
    # and last
    T4_inflated = t[get_T4_index(t, lc_inflated)]

    ## calculate the prediction based on our analytic formula
    dT1_predicted = abs(predict_dT1(dR, v_orb)) # [s]
    dT4_predicted = 0. # [s]
    # and what it is based on the models
    dT1_models = abs(T1_inflated - T1_uniform)*24.*60.*60. # [s]
    dT4_models = abs(T4_inflated - T4_uniform)*24.*60.*60. # [s]
    return dT1_predicted, dT1_models, dT4_predicted, dT4_models

In [ ]:
# set test parameters
inflation_testvals = np.array([0., 0.5, 1.0, 1.5, 2.0])
dT1_predicted_vals = np.zeros(len(inflation_testvals))
dT1_models_vals = np.zeros(len(inflation_testvals))
dT4_predicted_vals = np.zeros(len(inflation_testvals))
dT4_models_vals = np.zeros(len(inflation_testvals))
# run tests
for i_loop, inflate_val in enumerate(inflation_testvals):
    predicted1, modeled1, predicted4, modeled4 = test_dT1(inflate_val)
    dT1_predicted_vals[i_loop] = predicted1
    dT1_models_vals[i_loop] = modeled1
    dT4_predicted_vals[i_loop] = predicted4
    dT4_models_vals[i_loop] = modeled4

In [ ]:
# plot results
fig, ax = plt.subplots(figsize=(6,4))
fig.suptitle('First contact point', y=0.93)
ax.plot(inflation_testvals, dT1_predicted_vals, c='green', label='Predicted')
ax.plot(inflation_testvals, dT1_models_vals, c='black', marker='o', ls='--', label='Modeled')
ax.text(0.75, 0.05, 'inc = %.1f deg'%(inc), transform=ax.transAxes)
ax.set_xlabel('dR / H [scale heights]')
ax.set_ylabel('$\Delta T_1$ [s]')
ax.legend(loc='best')
plt.show()

fig, ax = plt.subplots(figsize=(6,4))
fig.suptitle('Last contact point', y=0.93)
ax.plot(inflation_testvals, dT4_predicted_vals, c='green', label='Predicted')
ax.plot(inflation_testvals, dT4_models_vals, c='black', marker='o', ls='--', label='Modeled')
ax.text(0.75, 0.05, 'inc = %.1f deg'%(inc), transform=ax.transAxes)
ax.set_xlabel('dR / H')
ax.set_ylabel('$\Delta T_4$ [s]')
ax.legend(loc='best')
plt.show()

Excellent agreement!

At least, there is excellent agreement when the inclination is 90 degrees (i.e. an equatorial orbit). As the inclination grows, so the planet's transit is elevated with respect to the stellar equator, there is larger disagreement at larger inflation factors due to geometric effects. When the asymmetry is larger and the planet is further from the stellar equator, $|\Delta T_1|$ is relatively larger than the agnostic prediction. We need to include an extra factor to account for that.

When the planet's orbit is along the stellar equator, the change in path length for first contact due to going from a morning limb radius of $R_0$ to $R_0 + \Delta R$ is simply $\delta x = \Delta R$. However, when elevated from the stellar equator, it takes a more complicated form of
$$
\Delta x =  \sqrt{ \left( R_\star + R_0 + \Delta R \right)^2 - \left( b R_\star \right)^2} - \sqrt{ \left( R_\star + R_0 \right)^2 - \left( b R_\star \right)^2}.
$$
Here, $R_\star$ is the stellar radius and $b$ is the impact parameter.

Let's test this:

In [ ]:
def get_impact_param(aRs, inc):
    b = aRs*np.cos(inc*(np.pi/180.))
    return b

def predict_dT1_general(R0, dR, Rs, b, vp):
    x_new = np.sqrt((Rs + R0 + dR)**2 - (b*Rs)**2)
    x_old = np.sqrt((Rs + R0)**2 - (b*Rs)**2)
    dx = x_new - x_old
    dT = (dx / vp)
    return dT

In [ ]:
## set orbital params
aRs = 12.0
inc = 87.0
aparams.a = aRs
aparams.inc = inc
# calculate the impact parameter
b = get_impact_param(aRs, inc)

## start with uniform radii
Rp_init = Rp_meter # starting limb radii in [m]
rprs_init = Rp_meter / Rs_meter
aparams.rp = rprs_init
aparams.rp2 = rprs_init
# generate the corresponding light curve
lc_uniform = asymmodel.light_curve(aparams)
# get the estimated first contact point
T1_uniform = t[get_T1_index(t, lc_uniform)]
# and last
T4_uniform = t[get_T4_index(t, lc_uniform)]

## now inflate the morning limb radius
inflation = 1.5 # % by which to increase radius, relative to initial radius
dR = inflation*H
Rp_inflated = Rp_init + dR
rprs_inflated = (Rp_inflated) / Rs_meter
aparams.rp = rprs_init
aparams.rp2 = rprs_inflated
# generate the corresponding light curve
lc_inflated = asymmodel.light_curve(aparams)
# get the estimated first contact point
T1_inflated = t[get_T1_index(t, lc_inflated)]
# and last
T4_inflated = t[get_T4_index(t, lc_inflated)]

## calculate the prediction based on our analytic formula
dT1_predicted = abs(predict_dT1_general(Rp_init, dR, Rs_meter, b, v_orb)) # [s]
dT4_predicted = 0. # [s]
# and what it is based on the models
dT1_models = abs(T1_inflated - T1_uniform)*24.*60.*60. # [s]
dT4_models = abs(T4_inflated - T4_uniform)*24.*60.*60. # [s]

In [ ]:
print('T1 test with general impact parameter -----')
print('Rp init = %.1f m'%(Rp_init))
print('dR = %.1f m (%.2f percent of Rp init)'%(dR, (dR/Rp_init)*100.))
print('inflated morning Rp = %.1f m'%(Rp_inflated))
print('---')
print('Predicted dT1 = %.2f s'%(dT1_predicted))
print('Modeled dT1 = %.2f s'%(dT1_models))
dT1_ratio = dT1_models / dT1_predicted
print('     Modeled / Predicted = %.3f'%(dT1_ratio))
print('Predicted dT4 = %.2f s'%(dT4_predicted))
print('Modeled dT4 = %.2f s'%(dT4_models))
# dT4_ratio = dT4_models / dT4_predicted
# print('     Modeled / Predicted = %.3f'%(dT4_ratio))

This seems to do a decent job! Let's run the same loop of tests.

In [ ]:
def test_dT1_general(inclination, inflation):
    ## set orbital params
    inc = inclination
    aparams.a = aRs
    aparams.inc = inc
    # calculate the impact parameter
    b = get_impact_param(aRs, inc)

    ## start with uniform radii
    Rp_init = Rp_meter # starting limb radii in [m]
    rprs_init = Rp_meter / Rs_meter
    aparams.rp = rprs_init
    aparams.rp2 = rprs_init
    # generate the corresponding light curve
    lc_uniform = asymmodel.light_curve(aparams)
    # get the estimated first contact point
    T1_uniform = t[get_T1_index(t, lc_uniform)]
    # and last
    T4_uniform = t[get_T4_index(t, lc_uniform)]

    ## now inflate the morning limb radius
    #inflation = 0.06 # % by which to increase radius, relative to initial radius
    dR = inflation*H
    Rp_inflated = Rp_init + dR
    rprs_inflated = (Rp_inflated) / Rs_meter
    aparams.rp = rprs_init
    aparams.rp2 = rprs_inflated
    # generate the corresponding light curve
    lc_inflated = asymmodel.light_curve(aparams)
    # get the estimated first contact point
    T1_inflated = t[get_T1_index(t, lc_inflated)]
    # and last
    T4_inflated = t[get_T4_index(t, lc_inflated)]

    ## calculate the prediction based on our analytic formula
    dT1_predicted = abs(predict_dT1_general(Rp_init, dR, Rs_meter, b, v_orb)) # [s]
    dT4_predicted = 0. # [s]
    # and what it is based on the models
    dT1_models = abs(T1_inflated - T1_uniform)*24.*60.*60. # [s]
    dT4_models = abs(T4_inflated - T4_uniform)*24.*60.*60. # [s]
    return dT1_predicted, dT1_models, dT4_predicted, dT4_models

In [ ]:
# set test parameters
inclination_testvals = np.array([90., 89., 88., 87.])
inflation_testvals = np.array([0., 0.5, 1.0, 1.5, 2.0])
dT1_predicted_vals2 = np.zeros((len(inclination_testvals), len(inflation_testvals)))
dT1_models_vals2 = np.zeros((len(inclination_testvals), len(inflation_testvals)))
dT4_predicted_vals2 = np.zeros((len(inclination_testvals), len(inflation_testvals)))
dT4_models_vals2 = np.zeros((len(inclination_testvals), len(inflation_testvals)))
# run tests
for inc_loop, inc_val in enumerate(inclination_testvals):
    for inf_loop, inflate_val in enumerate(inflation_testvals):
        predicted1, modeled1, predicted4, modeled4 = test_dT1_general(inc_val, inflate_val)
        dT1_predicted_vals2[inc_loop, inf_loop] = predicted1
        dT1_models_vals2[inc_loop, inf_loop] = modeled1
        dT4_predicted_vals2[inc_loop, inf_loop] = predicted4
        dT4_models_vals2[inc_loop, inf_loop] = modeled4

In [ ]:
# plot results
fig, ax = plt.subplots(figsize=(6,4))
fig.suptitle('first contact point', y=0.93)
# set of lines for each inclination ...
for inc_set, inc_val in enumerate(inclination_testvals):
    ax.plot(inflation_testvals, dT1_predicted_vals2[inc_set,:], c='green', label='Predicted')
    ax.plot(inflation_testvals, dT1_models_vals2[inc_set,:], c='black', marker='o', ls='--', label='Modeled')
ax.set_xlabel('dR / H')
ax.set_ylabel('$\Delta T_1$ [s]')
ax.legend(loc='best')
plt.show()

# plot results
fig, ax = plt.subplots(figsize=(6,4))
fig.suptitle('last contact point', y=0.93)
# set of lines for each inclination ...
for inc_set, inc_val in enumerate(inclination_testvals):
    ax.plot(inflation_testvals, dT4_predicted_vals2[inc_set,:], c='green', label='Predicted')
    ax.plot(inflation_testvals, dT4_models_vals2[inc_set,:], c='black', marker='o', ls='--', label='Modeled')
ax.set_xlabel('dR / H')
ax.set_ylabel('$\Delta T_4$ [s]')
ax.legend(loc='best')
plt.show()

Excellent agreement!

Now, let's move closer to the case we're really interested in -- comparing an asymmetric limb planet to a uniform limb planet. Above, only the first contact point changes because only the morning limb changes. Now, we'll add in a uniform limb planet with radius equal to the RMS of the asymmetric limbed planet's radii, which has the effect that this uniform limb planet has the same full transit depth as the asymmetric limb planet. 

In this new case, the uniform radius is slightly smaller than the morning radius, and slightly larger than the evening radius. So, both the first and last contact points should shift according to the change in projected path length between the uniform radius and each asymmetric radius. Let's assume that this is half the change between the asymmetric limbs -- i.e. $\Delta R_{new} = \frac{1}{2} \Delta R_{init}$.

I've already written an equation for $\Delta T_1$, and under this assumption it applies also to $\Delta T_4$.

So, I'll run the same test, only now I'll effectively start with the "inflated" planet and calculate the "uniform" planet as the depth-equivalent described above.

In [ ]:
def test_dT1_general_new(inclination, inflation):
    ## set orbital params
    inc = inclination
    aparams.a = aRs
    aparams.inc = inc
    # calculate the impact parameter
    b = get_impact_param(aRs, inc)

    ## start with uniform radii
    Rp_init = Rp_meter # starting limb radii in [m]
    rprs_init = Rp_meter / Rs_meter
    ## now inflate the morning limb radius
    #inflation = input param ...  % by which to increase radius, relative to initial radius
    dR = inflation*H
    Rp_inflated = Rp_init + dR
    rprs_inflated = (Rp_inflated) / Rs_meter
    aparams.rp = rprs_init
    aparams.rp2 = rprs_inflated
    # generate the corresponding light curve
    lc_inflated = asymmodel.light_curve(aparams)
    # get the estimated first contact point
    T1_inflated = t[get_T1_index(t, lc_inflated)]
    # and last
    T4_inflated = t[get_T4_index(t, lc_inflated)]
    
    ## now compute the depth-equivalent uniform radii planet
    rprs_unif = np.sqrt(0.5*(rprs_init**2 + rprs_inflated**2))
    aparams.rp = rprs_unif
    aparams.rp2 = rprs_unif
    # generate the corresponding light curve
    lc_uniform = asymmodel.light_curve(aparams)
    # get the estimated first contact point
    T1_uniform = t[get_T1_index(t, lc_uniform)]
    # and last
    T4_uniform = t[get_T4_index(t, lc_uniform)]

    ## calculate the prediction based on our analytic formula
    dT1_predicted = (predict_dT1_general(Rp_init, 0.5*dR, Rs_meter, b, v_orb)) # [s]
    dT4_predicted = (predict_dT1_general(Rp_init, 0.5*dR, Rs_meter, b, v_orb)) # [s]
    # and what it is based on the models
    dT1_models = (T1_inflated - T1_uniform)*24.*60.*60. # [s]
    dT4_models = (T4_inflated - T4_uniform)*24.*60.*60. # [s]
    return dT1_predicted, dT1_models, dT4_predicted, dT4_models

In [ ]:
# set test parameters
inclination_testvals = np.array([90., 89., 88., 87.])
inflation_testvals = np.array([0., 0.5, 1.0, 1.5, 2.0])
dT1_predicted_vals3 = np.zeros((len(inclination_testvals), len(inflation_testvals)))
dT1_models_vals3 = np.zeros((len(inclination_testvals), len(inflation_testvals)))
dT4_predicted_vals3 = np.zeros((len(inclination_testvals), len(inflation_testvals)))
dT4_models_vals3 = np.zeros((len(inclination_testvals), len(inflation_testvals)))
# run tests
for inc_loop, inc_val in enumerate(inclination_testvals):
    for inf_loop, inflate_val in enumerate(inflation_testvals):
        predicted1, modeled1, predicted4, modeled4 = test_dT1_general_new(inc_val, inflate_val)
        dT1_predicted_vals3[inc_loop, inf_loop] = predicted1
        dT1_models_vals3[inc_loop, inf_loop] = modeled1
        dT4_predicted_vals3[inc_loop, inf_loop] = predicted4
        dT4_models_vals3[inc_loop, inf_loop] = modeled4

In [ ]:
# plot results
print('TEST 3 -- inflated limb planet vs. equivalent depth uniform limb planet')
fig, ax = plt.subplots(figsize=(6,4))
fig.suptitle('first contact point', y=0.93)
# set of lines for each inclination ...
for inc_set, inc_val in enumerate(inclination_testvals):
    ax.plot(inflation_testvals, dT1_predicted_vals3[inc_set,:], c='green', label='Predicted')
    ax.plot(inflation_testvals, dT1_models_vals3[inc_set,:], c='black', marker='o', ls='--', label='Modeled')
ax.set_xlabel('percent increase of dR')
ax.set_ylabel('$\Delta T_1$ [s]')
ax.legend(loc='best')
plt.show()

# plot results
fig, ax = plt.subplots(figsize=(6,4))
fig.suptitle('last contact point', y=0.93)
# set of lines for each inclination ...
for inc_set, inc_val in enumerate(inclination_testvals):
    ax.plot(inflation_testvals, dT4_predicted_vals3[inc_set,:], c='green', label='Predicted')
    ax.plot(inflation_testvals, dT4_models_vals3[inc_set,:], c='black', marker='o', ls='--', label='Modeled')
ax.set_xlabel('percent increase of dR')
ax.set_ylabel('$\Delta T_4$ [s]')
ax.legend(loc='best')
plt.show()

In [ ]:
## compare median of T1 and T4 to analytic prediction for timing bias
# just for 90 degree inclination
dT_medianof14 = np.median((dT1_models_vals3[0,:], dT4_models_vals3[0,:]), axis=0)

x_new_eff = np.sqrt((Rs_meter + Rp_meter + (0.45*inflation_testvals*H))**2 - (b*Rs_meter)**2)
x_old_eff = np.sqrt((Rs_meter + Rp_meter)**2 - (b*Rs_meter)**2)
dx_effective = (x_new_eff - x_old_eff) # [m]
dT_effective = dx_effective / v_orb # [s]

# plot results
fig, ax = plt.subplots(figsize=(6,4))
fig.suptitle('Timing bias', y=0.93)
# set of lines for each inclination ...
for inc_set, inc_val in enumerate(inclination_testvals):
    ax.plot(inflation_testvals, -dT_effective, c='green', label='Pred')
    ax.plot(inflation_testvals, dT_medianof14, c='black', marker='o', ls='--', label='Modeled')
ax.set_xlabel('percent increase of dR')
ax.set_ylabel('$\Delta t$ [s]')
ax.legend(loc='best')
plt.show()

In [ ]:
rprs_inflated

In [ ]:
rprs_init

In [ ]:
rprs_unif

Ignore below ....

In [ ]:
def get_impact_param(aRs, inc):
    b = aRs*np.cos(inc*(np.pi/180.))
    return b
    
# def analytic_T13_uniform(Rs, Rp, aRs, inc, vp):
#     b = get_impact_param(aRs, inc) # impact parameter
#     offaxis_factor = np.sqrt(1. - (b**2))
#     dx23 = 2.*Rs*offaxis_factor
#     dt23 = dx23 / vp
#     return dt23

# def analytic_T34_uniform(Rp, aRs, inc, vp):
#     b = get_impact_param(aRs, inc) # impact parameter
#     offaxis_factor = np.sqrt(1. - (b**2))
#     dx34 = 2.*Rp*offaxis_factor
#     dt34 = dx34 / vp
#     return dt34


def lit_T14(Rs, Rp, aRs, inc, P):
    a = aRs*Rs # [m], assumes Rs is in [m]
    b = get_impact_param(aRs, inc)
    num = np.sqrt((Rs + Rp)**2 - (b*Rs)**2)
    angle = num/a
    pref = (P / np.pi)
    t14 = pref*np.arcsin(angle)
    return t14


In [ ]:
## functions to measure the points of transit contact
## ASSUMING UNIFORM LD LAW
def get_T1_index(time, y):
    # get the index where T1 occurs
    # defined as where model flux first goes < 1
    pretransit_idxs = np.where((y == 1.0) & (time < np.median(time)))[0] # 'oot' indexes
    T1_idx = pretransit_idxs[-1] 
    return T1_idx
def get_T4_index(time, y):
    # get the index where T4 occurs
    # defined as where model flux last goes = 1
    posttransit_idxs = np.where((y == 1.0) & (time > np.median(time)))[0] # 'oot' indexes
    T4_idx = posttransit_idxs[0] 
    return T4_idx
def get_T2_index(time, y, asymmetric='no'):
    # get index where T2 occurs 
    fulldepth = np.min(y) # full-depth is the minimum model flux
    if asymmetric == 'no':
        # -- works best for uniform limb LC
        # we assume uniform LD, so depth is same during entire full transit
        # then, T2 is point where depth switches from > full-depth to = full-depth
        fulltransit_idxs = np.where(y == fulldepth)[0]
        T2_idx = fulltransit_idxs[0] - 1
        return T2_idx
    elif asymmetric == 'yes':
        # -- works for asymmetry up to at least 20%
        ft_idx = np.array([])
        ref = float('%.4f'%(fulldepth))
        for i, yval in enumerate(y):
            cy = float('%.4f'%(yval))
            if cy == ref:
                ft_idx = np.append(ft_idx, i)
        T2_idx = int(ft_idx[0])
        return T2_idx

def get_T3_index(time, y, asymmetric='no'):
    # get index where T3 occurs 
    fulldepth = np.min(y) # full-depth is the minimum model flux
    if asymmetric == 'no':
        # -- works best for uniform limb LC
        # we assume uniform LD, so depth is same during entire full transit
        # then, T3 is point where depth switches from = full-depth to > full-depth
        fulltransit_idxs = np.where(y == fulldepth)[0]
        T3_idx = fulltransit_idxs[-1] 
        return T3_idx
    elif asymmetric == 'yes':
        # -- works for asymmetry up to at least 20%
        ft_idx = np.array([])
        ref = float('%.4f'%(fulldepth))
        for i, yval in enumerate(y):
            cy = float('%.4f'%(yval))
            if cy == ref:
                ft_idx = np.append(ft_idx, i)
        T3_idx = int(ft_idx[-1])
        return T3_idx

In [ ]:
## Comparing this to a batman model ----
comp_lc = unifmodel.light_curve(uparams)

# get model contact points
T1guess = t[get_T1_index(t, comp_lc)]
T2guess = t[get_T2_index(t, comp_lc, asymmetric='no')]
T3guess = t[get_T3_index(t, comp_lc, asymmetric='no')]
T4guess = t[get_T4_index(t, comp_lc)]
# compute model-derived durations
T14_model = T4guess - T1guess
T12_model = T2guess - T1guess
T34_model = T4guess - T3guess
T23_model = T3guess - T2guess
T13_model = T3guess - T1guess

print('Model values --')
print('T13 = %.2f min'%(T13_model*24.*60.))
print('T34 = %.2f min'%(T34_model*24.*60.))
print('T14 = %.2f min'%(T14_model*24.*60.))
# print('T12 = %.2f min'%(T12_model*24.*60.))
# print('T34 = %.2f min'%(T34_model*24.*60.))


# compute corresponding analytic-formula-derived durations in [s]
T14_lit = lit_T14(Rs_meter, Rp_meter, aRs, inc, per) #[day]
# # convert to [day] like model values

# # compute corresponding contact points
# T2_analy = -0.5*T23_analy
# T3_analy = 0.5*T23_analy
#T1_analy = -0.5*T14_analy
#T2_analy = T1_analy + T12_analy
#T4_analy = 0.5*T14_analy
#T3_analy = T4_analy - T34_analy


print('Analytic predictions --')
# print('T13 = %.2f min'%(T13_analy*24.*60.))
# print('T34 = %.2f min'%(T34_analy*24.*60.))
print('lit T14 = %.2f min'%(T14_lit*24.*60.))
#print('T12 = %.2f min'%(T12_analy*24.*60.))
#print('T34 = %.2f min'%(T34_analy*24.*60.))

# compare
#T14_ratio = T14_model / T14_analy
#T12_ratio = T12_model / T12_analy
#T34_ratio = T34_model / T34_analy

print('Model value / Analytic value ---')
#print('T14 ratio = %.3f'%(T14_ratio))
#print('T12 ratio = %.3f'%(T12_ratio))
#print('T34 ratio = %.3f'%(T34_ratio))

fig, ax = plt.subplots(figsize=(10,4))
ax.axvline(T1guess, c='blue', lw=0.5, alpha=0.3)
ax.axvline(T2guess, c='blue', lw=0.5, alpha=0.3)
ax.axvline(T3guess, c='blue', lw=0.5, alpha=0.3)
ax.axvline(T4guess, c='blue', lw=0.5, alpha=0.3)
#ax.axvline(T1_analy, c='orange', lw=0.5, alpha=0.3)
# ax.axvline(T2_analy, c='orange', lw=0.5, alpha=0.3)
# ax.axvline(T3_analy, c='orange', lw=0.5, alpha=0.3)
#ax.axvline(T4_analy, c='orange', lw=0.5, alpha=0.3)
#ax.axvline(0.5*T14_analy, c='purple', lw=0.5, alpha=0.4)
#ax.axvline(-0.5*T14_analy, c='purple', lw=0.5, alpha=0.4)
ax.plot(t, comp_lc, c='black')
plt.show()

In [ ]:
T14_lit